# BackTesting 


In [12]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data = data.loc['2020-01-01':]
data

,Open,High,Low,Close,Adj Close,Volume,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997,7275.276595,7290.616103,7572.187445,8224.260889,45.182876,-61.114299,-85.231483,6985.470215,0
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465,7264.120752,7261.554590,7549.178926,8211.934613,39.271849,-79.166603,-84.018507,7344.884277,1
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032,7267.215739,7269.490751,7541.167371,8203.307247,50.861068,-63.736756,-79.962157,7410.656738,1
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275,7265.960710,7282.935130,7536.049307,8195.420177,52.642173,-45.674693,-73.104664,7411.317383,1
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095,7261.438070,7295.162012,7531.157859,8187.618159,52.660734,-30.950295,-64.673790,7769.219238,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-16,104293.578125,107780.578125,103322.984375,106029.718750,106029.718750,91020417816,97501.756771,98891.717554,90677.438970,72849.742753,69.192878,3707.914747,3675.381559,106140.601562,1
2024-12-17,106030.687500,108268.445312,105291.734375,106140.601562,106140.601562,68589364868,98044.915104,99582.087460,91283.837503,73180.995080,69.336619,3847.987439,3709.902735,100041.539062,0
2024-12-18,106147.296875,106470.609375,100041.539062,100041.539062,100041.539062,93865656139,98361.545052,99625.844755,91627.276780,73448.264174,54.322688,3427.344393,3653.391067,97490.953125,0


In [19]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, plot_heatmaps
import talib 

class SMAcross(Strategy):
    def init(self):
        self.ema = self.I(talib.EMA, self.data.Close, timeperiod=20)
                
    def next(self): # Se llama en cada paso o dia.
        price = self.data.Close[-1] # Precio de cierre del ultimo dia, ya que viene el acumulado por cada dia que pasa.

        if price > self.ema[-1] and not self.position.is_long: 
            # self.buy(size=10, tp=price*1.3, sl=price*0.85)
            self.buy(sl=price*0.95)
        elif price <= self.ema[-1] and self.position.is_long:
            self.position.close()

bt = Backtest(data, SMAcross,cash=200000 )
bt.run()

Start                     2020-01-01 00:00:00
End                       2024-12-20 00:00:00
Duration                   1815 days 00:00:00
Exposure Time [%]                   59.801762
Equity Final [$]               1871296.116309
Equity Peak [$]                2035758.335059
Return [%]                         835.648058
Buy & Hold Return [%]             1257.688375
Return (Ann.) [%]                   56.741883
Volatility (Ann.) [%]               69.128496
Sharpe Ratio                         0.820818
Sortino Ratio                         2.15424
Calmar Ratio                         1.111572
Max. Drawdown [%]                    -51.0465
Avg. Drawdown [%]                   -7.418932
Max. Drawdown Duration      860 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                  101
Win Rate [%]                        27.722772
Best Trade [%]                      67.130993
Worst Trade [%]                     -5.042587
Avg. Trade [%]                    

In [20]:
bt.plot()

c:\Users\Ivo\miniconda3\envs\backtest_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Ivo\miniconda3\envs\backtest_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Ivo\miniconda3\envs\backtest_env\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
c:\Users\Ivo\miniconda3\envs\backtest_env\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\Ivo\miniconda3\envs\backtest_en

GridPlot(id='p3060', ...)